# Regressions to predict House Prices 

You can check the data description [here](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt)

In [ ]:
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import skew 


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points




Let's load the train and test set.

In [ ]:
train = pd.read_csv('housetrain.csv')
train.shape

In [ ]:
test = pd.read_csv('housetest.csv')
test.shape

In [ ]:
train.head()

Realationship of numerical feature with target:

In [ ]:
plt.scatter(train['GrLivArea'], train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)

We can see there's strong linear relationship between `GrLivArea` and the target. This will be important feature.

Relationship of categorical feature with target:

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=train['OverallQual'], y=train["SalePrice"])

Again, good strong relationship of 'OverallQual' with target as expected.

### Correlation

The best way to get quick overview of the reelationships between features is using correlation matrix.

In [ ]:
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

Let's zoom into the correlation matrix and only show top 10 features correlation with 'SalePrice'

In [ ]:
cols = corrmat.nlargest(10, 'SalePrice').loc[:, 'SalePrice'].index

plt.figure(figsize=(10,5))
sns.heatmap(train[cols].corr(), annot=True)

We can see that 'OverallQual' and 'GrLivArea' are highly correlated with 'SalePrice'

# Data Processing

## Preprocessing:

- Imputing missing values

- Transforming some numerical variables that seem really categorical

-  Log-transformation of skewed features

- Getting dummy variables for categorical features. 



## Outliers

Let's explore outliers in `GrLivArea`


In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()


There are one extremely large `GrLivArea` that are of a low price. This value are extreme oultlier. Therefore, we can safely remove it.

**Task:**
- Remove the outlier

In [ ]:
## your code

Outliers removal is nit always safe.  We decided to delete these two as they are very huge and  really  bad ( extremely large areas for very low  prices).

Now, let's do some transformation on our data, to do this we need to concatenate our train and test set.

In [ ]:
split_index = train.shape[0] # get the splitting point to split the data later
full_data = pd.concat([train, test])
full_data.shape

## Target Variable

**SalePrice** is the variable we need to predict. So let's visualize the distribution of this variable.

In [ ]:
sns.distplot(full_data['SalePrice'])

The target variable is right skewed.  As linear models love normally distributed data , it's a good idea to transform this variable and make it more normally distributed.

 **Log-transformation of the target variable**

In [ ]:
#We use the numpy fuction log which  applies log(x) to all elements of the column
full_data["SalePrice"] = np.log1p(full_data["SalePrice"])

# Check the new distribution 
sns.distplot(full_data['SalePrice'])


The skew seems now corrected and the data appears more normally distributed. Let's look at `GrLivArea`

In [ ]:
sns.distplot(full_data['GrLivArea'])

'GrLivArea' also skewed, let's normalize it

In [ ]:
full_data['GrLivArea'] = np.log1p(full_data['GrLivArea'])
sns.distplot(full_data['GrLivArea'])

We can check the skewness of each numerical feature with scipy `skew` function

In [ ]:
numeric_feats = full_data.dtypes[full_data.dtypes != "object"].index
skewness = abs(full_data[numeric_feats].dropna().apply(skew))
skewness.sort_values(ascending=False)

We'll do log transformation for features with skewness more than 0.75.

**Task:**
- Apply the log transformation as above to all numerical features that are skewed.
- Use `skew` function from scipy to calculate the skewness of each feature.
- Apply log transformation (`np.log1p`) for every features that have skewness value more than 0.75. The skewness value should be in absolute value. i.e: `abs(skewness)`. Use `dropna` to remove missing values on the feature before calculating the skewness.

In [ ]:
## your code

### Missing Data

Let's check the percentage of missing data for each column.

**Task:**

- create a dataframe with columns name as index.
- the dataframe has one column, `Missing percentage` which is the percentage of missing values for each column.
- only show columns with missing values.

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=data_na.index, y=data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

**Task:**
- Split the input features and target.

In [ ]:
## your code

**Transforming some numerical variables that are really categorical**

**Task:**
- Transform column 'MSSubClass', 'OverallCond', 'YrSold' and 'MoSold' into categorical feature.

In [ ]:
## your code

### Imputing missing values 

Take note that for categorical variables, NA here does not mean a missing value, but should be treated as a separate category. For numerical data impute the missing values with mean of the feature.

**Task:**

- Apply one-hot-encoding to all categorical features.
- Impute missing values for numerical features.
- Make sure there are no more missing values in the data.

In [ ]:
## your code

In [ ]:
X.shape

## Split the data

Now we can split the data back into train and test set. 

In [ ]:
X_train = X.iloc[:split_index, :]
X_test = X.iloc[split_index:, :]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

In [ ]:
print(X_train.shape)
print(X_test.shape)

# Modelling

**Task:**

- Evaluate Linear Regression, Lasso, Ridge, Random Forest and Gradient Boosting using cross-validation with default parameters.
- Tune the parameters of the models using GridSearchCV. Does the score improve?
- Use RMSE (root mean squared error) as the evaluation metric.

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error

In [ ]:
## cross-validation with rmse scoring function

### Linear Regression

### Ridge Regression

### Lasso

### Random Forest

### Gradient Boosting